# Exploratory Data Analysis of DICOM datasets

> _note that we are not aiming for optimal memory use here in some cases. You should consider general best practices if writing optimized code_

Let's load some images

In [ ]:
import pydicom
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy.ma as ma
import numpy as np
import os

plt.rcParams["figure.figsize"] = (10,10)

path = f"samples\DICOM CT study\JohnDoe_CT_series"
slices = [pydicom.dcmread(os.path.join(path, f)) for f in sorted(os.listdir(path))]

In [ ]:
# How many slices do we have?

len(slices)

In [ ]:
# Let's take a look at some metadata. Note slice thickness, dimensions and pixel spacing

print(slices[0])

In [ ]:
# Let's print some of those data elements using PyDicom's member variable intercepts:

print(f"Pixel Spacing: {slices[0].PixelSpacing}")
print(f"Slice Thickness: {slices[0].SliceThickness}mm")

In [ ]:
# Another way to address data elements in pyDicom (e.g. if we are dealing with those that are not 
# in the DICOM data dictionary):

print(f"Pixel Spacing: {slices[0][0x0028,0x0030].value}")

In [ ]:
# Let's extract the pixel data

image_data = np.stack([s.pixel_array for s in slices])

In [ ]:
# Dimensions?

image_data.shape

In [ ]:
# Data type?

image_data.dtype

In [ ]:
# Let's visualize a slice
# Even by looking at the array shape we can tell that axis 0 is the z axis 
# and if we slice across it we will get axial slices

img = image_data[115,:,:]

plt.imshow(img,cmap = "gray")

In [ ]:
# Not bad! Let's visualize a bunch of slices

fig, ax = plt.subplots(5, 5, figsize=[10,10])

for i in range(25):
    ix = i*int(len(slices)/25)
    ax[int(i/5), int(i%5)].set_title(f"slice {ix}")
    ax[int(i/5), int(i%5)].imshow(image_data[ix, :, :], cmap='gray')
    ax[int(i/5), int(i%5)].axis("off")

plt.show()

In [ ]:
# Let's do coronal slice now

img_coronal = image_data[:,250,:]
plt.imshow(img_coronal, cmap="gray")

Whoa, what just happened?
Remember that our voxels are not isotropic? We need to scale along Z dimension to get proper representation


In [ ]:
aspect_ratio = slices[0].SliceThickness/slices[0].PixelSpacing[0]

plt.imshow(img_coronal, cmap="gray", aspect = aspect_ratio)

In [ ]:
# Let's crop our extracted slice a little to focus on the anatomy

img_crop = img[110:400,:]
plt.imshow(img_crop,cmap = cm.Greys_r)

In [ ]:
# Let's visualize the histogram - what values do we have here?

p = img_crop.flatten()
# vals, bins, ignored = plt.hist(p[p>-900], bins = 200)
vals, bins, ignored = plt.hist(p, bins = 200)
plt.show()

In [ ]:
print(img_crop.max())
print(img_crop.min())

### A reminder on Hounsfield Units
The Hounsfield scale is used to measure radiodensity and, in reference to medical-grade CT scans, can provide an accurate absolute density for the type of tissue depicted. 

https://en.wikipedia.org/wiki/Hounsfield_scale

<img src= "img/hu_j.png" width=400>

Looks good, but what is this extreme stuff? What's 3071 and -3024?

In [ ]:
# Let's look at the bottom end:

np.sort(np.unique(img_crop))

So we can see that after a bunch of pixels valued at -3024 we immediately jump to -1024 which wits quite close to the range of the Hounsfield scale. This could lead you to believe that -3024 is not a result of reconstruction of the CT sinogram, but rather something that was added to the image later. And it is so, and it happens quite often - CT scanners add "magic" values to pad volumes to rectangles (since images are saved as rectangular grids of voxels). A large negative value is selected so that it does not overlap with values that would be legitimately present on an HU scale.

Now let's look at the upper end of that range

In [ ]:
# Let us apply the window

hu_min = 2000
hu_max = 4000

# Note that here we are using the Numpy's Masked Array module which allows us to mask values that fall outside of 
# range that interests us
# windowed_img = ma.masked_where((img_crop < hu_min) | (img_crop > hu_max), img_crop)
windowed_img = np.copy(img_crop)
windowed_img[np.where(windowed_img < hu_min)] = hu_min
windowed_img[np.where(windowed_img > hu_max)] = hu_max

# plt.imshow(ma.masked_array(data = img_crop, mask = img_crop > hu_max, fill_value = img_crop.min()).filled(), cmap = cm.Greys_r)
plt.imshow(windowed_img, cmap="gray")

In [ ]:
windowed_img.shape

In [ ]:
# Let's zoom in on this thing a bit better

plt.imshow(img_crop[15:60, 380:430], cmap="gray")

In [ ]:
# Using numpy's Masked Array for convenience

masked = ma.masked_outside(img_crop, hu_min, hu_max)

np.sort(np.unique(img_crop[~masked.mask]))

In [ ]:
# Let us see what values we have in our windowed range

_ = plt.hist(masked.flatten(), bins = 15)

plt.show()

That has to be a synthetic material. And it is - that is a chemotherapy port.

An interesting observation: original image's voxel range:

In [ ]:
print(f"range: {img_crop.max()-img_crop.min()}")

However, we don't really need the extreme values. As you've seen we can safely ditch anything below -1200 and anything over 1300 and still have all the information about anatomy. It might be safe to apply such mask to the entire dataset. Given that you will be converting your data to floating point format for ML workflows, you will be able to use much smaller scale factor for conversion which in this case would effectively double the precision at which your model can operate while keeping the same memory footprint.